# Molecular Docking - Mahkota Dewa vs Diabetic Nephropathy Targets

This notebook performs automated molecular docking using AutoDock Vina.

**Pre-prepared files required:**
- `receptors/` - 5 receptor PDBQT files
- `ligands/` - 13 drug-like compound PDBQT + 4 controls
- `binding_sites.json` - Auto-extracted binding site coordinates

**Workflow:**
1. Upload ZIP with prepared files
2. Run docking (69 jobs)
3. Analyze and download results

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q vina pandas numpy tqdm
print("✅ Dependencies installed successfully!")

## 2. Configuration

In [ ]:
# ============================================
# CONFIGURATION - Mahkota Dewa DN Study
# ============================================

PROJECT_NAME = "mahkota_dewa_dn"

# Target proteins and their PDB IDs
TARGET_PROTEINS = {
    "6MS7": "PPARG",   # PPAR-gamma
    "1HWK": "HMGCR",   # HMG-CoA reductase
    "4YAY": "AGTR1",   # Angiotensin II receptor 1
    "1TBF": "PDE5A",   # Phosphodiesterase 5A
    "3QXY": "RELA",    # NF-kB p65
}

# Control compounds and their specific targets
CONTROL_MAPPINGS = {
    "Pioglitazone": "6MS7",   # PPARG agonist
    "Atorvastatin": "1HWK",   # HMGCR inhibitor
    "Losartan": "4YAY",       # AGTR1 blocker
    "Sildenafil": "1TBF",     # PDE5A inhibitor
}

# 13 Drug-like compounds (passed ADMET)
DRUGLIKE_COMPOUNDS = [
    "Gallic_acid",
    "Quercetin",
    "Kaempferol",
    "Phalerin",
    "264-trihydroxy-4-methoxybenzophenone",
    "Swertianin",
    "Apigenin",
    "Luteolin",
    "Caffeic_acid",
    "Pinoresinol",
    "Lariciresinol",
    "Matairesinol",
    "Naringeninchalcone",
]

# Docking parameters
EXHAUSTIVENESS = 8  # 8-32, higher = more thorough but slower
NUM_MODES = 9       # Number of binding modes to generate

print(f"Project: {PROJECT_NAME}")
print(f"Targets: {list(TARGET_PROTEINS.values())}")
print(f"Test compounds: {len(DRUGLIKE_COMPOUNDS)}")
print(f"Control compounds: {len(CONTROL_MAPPINGS)}")
print(f"\nDocking jobs: {len(DRUGLIKE_COMPOUNDS) * len(TARGET_PROTEINS) + len(CONTROL_MAPPINGS)}")

## 3. Upload Prepared Files

In [ ]:
from google.colab import files
import zipfile
import os
import json

# Create working directories
os.makedirs("receptors", exist_ok=True)
os.makedirs("ligands", exist_ok=True)
os.makedirs("results", exist_ok=True)

print("Upload your docking_files.zip containing:")
print("  - receptors/*.pdbqt")
print("  - ligands/*.pdbqt")
print("  - binding_sites.json")
print()

uploaded = files.upload()

# Extract ZIP
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"\n✅ Extracted {filename}")

In [ ]:
# Verify uploaded files
import glob

receptor_files = glob.glob("receptors/*.pdbqt")
ligand_files = glob.glob("ligands/*.pdbqt")

print(f"📁 Receptors found: {len(receptor_files)}")
for f in receptor_files:
    print(f"   - {os.path.basename(f)}")

print(f"\n📁 Ligands found: {len(ligand_files)}")
for f in ligand_files:
    print(f"   - {os.path.basename(f)}")

# Load binding sites
with open("binding_sites.json", "r") as f:
    binding_sites = json.load(f)

print(f"\n📍 Binding sites loaded: {len(binding_sites)}")
for pdb_id, site in binding_sites.items():
    print(f"   {pdb_id}: center=({site['center_x']:.1f}, {site['center_y']:.1f}, {site['center_z']:.1f}), "
          f"size=({site['size_x']:.1f} × {site['size_y']:.1f} × {site['size_z']:.1f}) Å")

## 4. Run Molecular Docking

In [ ]:
from vina import Vina
from tqdm import tqdm
import pandas as pd

def run_docking(receptor_pdbqt, ligand_pdbqt, center, size, output_pdbqt):
    """Run Vina docking with specified binding site."""
    try:
        v = Vina(sf_name='vina')
        v.set_receptor(receptor_pdbqt)
        v.set_ligand_from_file(ligand_pdbqt)
        
        v.compute_vina_maps(
            center=center,
            box_size=size
        )
        
        v.dock(exhaustiveness=EXHAUSTIVENESS, n_poses=NUM_MODES)
        
        # Get best score
        energies = v.energies()
        best_score = energies[0][0] if energies is not None and len(energies) > 0 else None
        
        # Save poses
        v.write_poses(output_pdbqt, n_poses=NUM_MODES)
        
        return best_score
        
    except Exception as e:
        print(f"Docking error: {e}")
        return None

In [ ]:
# Build job list
jobs = []

# 1. Test compounds vs all targets (13 × 5 = 65 jobs)
for compound in DRUGLIKE_COMPOUNDS:
    ligand_file = f"ligands/{compound}.pdbqt"
    if not os.path.exists(ligand_file):
        print(f"⚠️ Ligand not found: {ligand_file}")
        continue
    
    for pdb_id, gene in TARGET_PROTEINS.items():
        receptor_file = f"receptors/{pdb_id}.pdbqt"
        if not os.path.exists(receptor_file):
            print(f"⚠️ Receptor not found: {receptor_file}")
            continue
        
        jobs.append({
            "compound": compound,
            "target": gene,
            "pdb_id": pdb_id,
            "receptor_file": receptor_file,
            "ligand_file": ligand_file,
            "type": "test"
        })

# 2. Control compounds vs their specific targets (4 jobs)
for compound, pdb_id in CONTROL_MAPPINGS.items():
    ligand_file = f"ligands/{compound}.pdbqt"
    receptor_file = f"receptors/{pdb_id}.pdbqt"
    
    if os.path.exists(ligand_file) and os.path.exists(receptor_file):
        jobs.append({
            "compound": compound,
            "target": TARGET_PROTEINS[pdb_id],
            "pdb_id": pdb_id,
            "receptor_file": receptor_file,
            "ligand_file": ligand_file,
            "type": "control"
        })

print(f"\n📋 Total docking jobs: {len(jobs)}")
print(f"   - Test compounds: {sum(1 for j in jobs if j['type'] == 'test')}")
print(f"   - Control compounds: {sum(1 for j in jobs if j['type'] == 'control')}")

In [ ]:
# Run docking
results = []

print(f"\n🧪 Running {len(jobs)} docking simulations...\n")

for job in tqdm(jobs):
    pdb_id = job["pdb_id"]
    site = binding_sites.get(pdb_id, {})
    
    # Get binding site parameters
    center = [
        site.get("center_x", 0),
        site.get("center_y", 0),
        site.get("center_z", 0)
    ]
    size = [
        site.get("size_x", 25),
        site.get("size_y", 25),
        site.get("size_z", 25)
    ]
    
    # Output file
    output_file = f"results/{job['target']}_{job['compound']}_docked.pdbqt"
    
    # Run docking
    score = run_docking(
        job["receptor_file"],
        job["ligand_file"],
        center,
        size,
        output_file
    )
    
    results.append({
        "compound": job["compound"],
        "target": job["target"],
        "pdb_id": pdb_id,
        "binding_affinity": score,
        "type": job["type"],
        "output_file": output_file if score else None
    })

print("\n✅ Docking complete!")

## 5. Analyze Results

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results)

# Filter successful docking
valid_results = results_df[results_df["binding_affinity"].notna()].copy()

print("="*60)
print("DOCKING RESULTS SUMMARY")
print("="*60)
print(f"\n✅ Successful: {len(valid_results)}/{len(results_df)}")

# Separate test and control
test_results = valid_results[valid_results["type"] == "test"].copy()
control_results = valid_results[valid_results["type"] == "control"].copy()

In [ ]:
# Best compounds per target
print("\n" + "="*60)
print("TOP 3 COMPOUNDS PER TARGET")
print("="*60)

for pdb_id, gene in TARGET_PROTEINS.items():
    print(f"\n📊 {gene} ({pdb_id}):")
    
    # Get control for this target
    control = control_results[control_results["pdb_id"] == pdb_id]
    if len(control) > 0:
        ctrl_score = control.iloc[0]["binding_affinity"]
        ctrl_name = control.iloc[0]["compound"]
        print(f"   🎯 Control: {ctrl_name} = {ctrl_score:.2f} kcal/mol")
    else:
        ctrl_score = None
    
    # Get top 3 test compounds
    target_results = test_results[test_results["pdb_id"] == pdb_id].sort_values("binding_affinity")
    print(f"   Top test compounds:")
    for i, (_, row) in enumerate(target_results.head(3).iterrows()):
        score = row["binding_affinity"]
        better = "✅" if ctrl_score and score < ctrl_score else ""
        print(f"   {i+1}. {row['compound']}: {score:.2f} kcal/mol {better}")

In [ ]:
# Overall ranking
print("\n" + "="*60)
print("OVERALL TOP 10 COMPOUND-TARGET PAIRS")
print("="*60)

overall_top = valid_results.sort_values("binding_affinity").head(10)
print(overall_top[["compound", "target", "binding_affinity", "type"]].to_string(index=False))

In [ ]:
# Create affinity matrix
print("\n" + "="*60)
print("BINDING AFFINITY MATRIX (kcal/mol)")
print("="*60)

# Pivot table
matrix = test_results.pivot_table(
    values="binding_affinity",
    index="compound",
    columns="target",
    aggfunc="first"
)

# Add average column
matrix["Average"] = matrix.mean(axis=1)
matrix = matrix.sort_values("Average")

print(matrix.round(2).to_string())

In [ ]:
# Save results
results_df.to_csv(f"results/{PROJECT_NAME}_docking_results.csv", index=False)
matrix.to_csv(f"results/{PROJECT_NAME}_affinity_matrix.csv")

# Summary statistics
summary = {
    "total_jobs": len(results_df),
    "successful": len(valid_results),
    "test_compounds": len(DRUGLIKE_COMPOUNDS),
    "control_compounds": len(CONTROL_MAPPINGS),
    "targets": list(TARGET_PROTEINS.values()),
    "best_overall": valid_results.sort_values("binding_affinity").head(5).to_dict("records")
}

with open(f"results/{PROJECT_NAME}_docking_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print(f"\n✅ Results saved:")
print(f"   - {PROJECT_NAME}_docking_results.csv")
print(f"   - {PROJECT_NAME}_affinity_matrix.csv")
print(f"   - {PROJECT_NAME}_docking_summary.json")

## 6. Download Results

In [ ]:
import shutil

# Create zip archive
shutil.make_archive(f"{PROJECT_NAME}_docking_results", "zip", "results")

# Download
files.download(f"{PROJECT_NAME}_docking_results.zip")
print("✅ Results downloaded!")